In [1]:
import sympy as sp

In [2]:
import matplotlib.pyplot as plt

In [3]:
import sympy.physics.units.quantities as sq

In [4]:
from sympy.physics.quantum.constants import hbar

In [5]:
PSI_FUNCTION = function( "psi" )( x )
POTENTIAL_FUNCTION = function( "V" )( x )
TOTAL_ENERGY_SYMBOL = var( 'E' )
MASS_SYMBOL = var( 'm' )
REDUCED_PLANCK_CONSTANT_SYMBOL = var( "hbar" )#sq.Symbol( "hbar" )
POSITION_SYMBOL = var( 'x' )#x#sp.Symbol( 'x', positive = True )

def time_independent_schroedinger_equation( 
            psi = PSI_FUNCTION, 
            potential = POTENTIAL_FUNCTION, 
            total_energy = TOTAL_ENERGY_SYMBOL, 
            mass = MASS_SYMBOL, 
            reduced_planck_constant = REDUCED_PLANCK_CONSTANT_SYMBOL, 
            position = POSITION_SYMBOL 
        ): 
    return -reduced_planck_constant ** 2 / 2 * mass \
                    * diff( psi, position, 2 ) \
                    + potential * psi == total_energy * psi


In [6]:
set_equal = lambda to_set, value : to_set == value
both_sides = lambda equation, opreation : opreation( equation.lhs() ) == opreation( equation.rhs() )
equation_to_dict = lambda equation : { equation.lhs() : equation.rhs() }

In [7]:
hbar_eq = sp.Eq( hbar, sp.physics.units.planck / ( 2 * sp.pi ) )

In [8]:
hbar_eq

Eq(hbar, planck/(2*pi))

In [9]:
psi = PSI_FUNCTION
potential = POTENTIAL_FUNCTION
total_energy = TOTAL_ENERGY_SYMBOL
mass = MASS_SYMBOL
#reduced_planck_constant = REDUCED_PLANCK_CONSTANT_SYMBOL
x = POSITION_SYMBOL

In [10]:
hb = var( "hbar" )

In [11]:
k = var( 'k' )

In [12]:
k_squared = k ** 2 == ( ( 2 * total_energy * mass ) / ( hb ** 2 ) )

In [13]:
k_squared

k^2 == 2*E*m/hbar^2

In [14]:
class ZeroPotential( sp.Function ): 
    @classmethod
    def eval( cls, position ):
        return 0

In [15]:
#https://docs.sympy.org/latest/modules/assumptions/index.html#querying
#https://docs.sympy.org/latest/modules/codegen.html#module-sympy.codegen.cxxnodes

class Potential( sp.Function ): 
    DEFAULT_POTENTIAL = var( "V_0" )#sp.Symbol( "V_0" )
    @classmethod
    def eval( cls, position, potential = DEFAULT_POTENTIAL ): 
        return potential

In [16]:
class TunnelPotential( sp.Function ): 
    DEFAULT_WELL_LENGTH = var( 'L' )#sp.Symbol( 'L' )
    DEFAULT_POTENTIAL = Potential.DEFAULT_POTENTIAL
    DEFAULT_START = 0
    @classmethod
    def eval( cls, position, length = DEFAULT_WELL_LENGTH, 
             start = DEFAULT_START, potential = DEFAULT_POTENTIAL ): 
        if position < start or position > sp.simplify( length + start ): 
            return ZeroPotential.eval( position )
        return Potential.eval( position, potential )

In [31]:
class Stepper: 

    LEFT = "LEFT", 
    RIGHT = "RIGHT"
    
    def __init__( self, first_step, new_steps = None ): 
        self.steps = new_steps if new_steps else []
        self.steps.append( first_step )
        self.chaining = False
    
    def step_number( self, step = None ): 
        return ( len( self.steps ) - 1 ) if not step else step
    
    def last_step( self, step = None ):
        return self.steps[ self.step_number( step ) ]
    
    def _return_chain( self, step, chain ): 
        return self if ( chain or self.chaining ) else step        
    
    def add_step( self, new_step, chain = False ):
        self.steps.append( new_step )
        return self._return_chain( self.steps[ -1 ], chain )

    def operate( self, operation, step = None, chain = False ):
        self.steps.append( operation( self.last_step( step ) ) )
        return self._return_chain( self.steps[ -1 ], chain )

    def manipulate( self, operation, chain = False ): 
        return self._return_chain( self.add_step( 
                both_sides( self.last_step(), operation ) ), 
                chain 
            )
    
    def delete_step( self, step ): 
        old_step = self.steps[ step ]
        del self.steps[ step ]
        return old_step
    
    def undo( self, step = None, chain = False ): 
        return self._return_chain( 
                self.delete_step( self.step_number( step ) ), 
                chain 
            )
    
    def clone( self, from_step = None ):
        return self.branch( lambda blank : blank, from_step )
    
    def branch( self, operation = None, from_step = None ): 
        from_step = self.step_number( from_step )
        return Stepper( 
                both_sides( self.last_step( from_step ), operation ), 
                self.steps[ : self.step_number( from_step ) : ], 
            )
    
    def substitute_constant( self, constant, chain = False ):
            return self.operate( lambda step : step.subs( { constant.rhs() : constant.lhs() } ) )
    
    def negate_add( self, side_to_negate_add, chain = False ): 
        last_step = self.last_step()
        return self.manipulate( 
                lambda side : side + -( 
                    last_step.lhs() if side_to_negate_add == Stepper.LEFT 
                            else last_step.rhs() 
                        ), 
                chain 
            )
    
    def rename( self, old_symbol, new_symbol_name, chain = False ): 
        last_step = self.last_step()
        assert old_symbol in last_step.atoms() or old_symbol in last_step.atoms( sp.Function )
        return self.add_step( last_step.subs( 
            old_symbol if type( old_symbol ) == dict \
                    else { old_symbol : new_symbol_name } ), chain 
                )
    
    def chain( self, operation ):
        self.chaining = True
        operation( self )
        self.chaining = False
        return self
    
    def begin_chain( self ): 
        self.chaining = True
        return self
    
    def end_chain( self ): 
        self.chaining = False
        return self


In [32]:
k_squared

k^2 == 2*E*m/hbar^2

In [33]:
psi_region = [
        Stepper( time_independent_schroedinger_equation( potential = 0 ) ), #ZeroPotential ) ), 
        Stepper( time_independent_schroedinger_equation( potential = Potential.DEFAULT_POTENTIAL ) ), 
        Stepper( time_independent_schroedinger_equation( potential =  0 ) ) #ZeroPotential ) )
    ]

In [34]:
psi_region[ 0 ].last_step()

-1/2*hbar^2*m*diff(psi(x), x, x) == E*psi(x)

In [35]:
psi_region[ 0 ].manipulate( lambda side : ( side - psi_region[ 0 ].last_step().rhs() ) )#.simplify() )

-1/2*hbar^2*m*diff(psi(x), x, x) - E*psi(x) == 0

In [36]:
psi_region[ 0 ].manipulate( lambda side : -( side / total_energy ).simplify() )

1/2*(hbar^2*m*diff(psi(x), x, x) + 2*E*psi(x))/E == 0

In [37]:
psi_region[ 0 ].manipulate( lambda side : side - psi )

1/2*(hbar^2*m*diff(psi(x), x, x) + 2*E*psi(x))/E - psi(x) == -psi(x)

In [38]:
psi_region[ 0 ].manipulate( lambda side : side * k_squared.rhs() )

E*m*((hbar^2*m*diff(psi(x), x, x) + 2*E*psi(x))/E - 2*psi(x))/hbar^2 == -2*E*m*psi(x)/hbar^2

In [39]:
psi_region[ 0 ].manipulate( lambda side : side / psi )

E*m*((hbar^2*m*diff(psi(x), x, x) + 2*E*psi(x))/E - 2*psi(x))/(hbar^2*psi(x)) == -2*E*m/hbar^2

In [40]:
k_squared

k^2 == 2*E*m/hbar^2

In [41]:
psi_region[ 0 ].substitute_constant( k_squared )

E*m*((hbar^2*m*diff(psi(x), x, x) + 2*E*psi(x))/E - 2*psi(x))/(hbar^2*psi(x)) == -2*E*m/hbar^2

In [42]:
psi_region[ 0 ].manipulate( lambda side : side * psi )

E*m*((hbar^2*m*diff(psi(x), x, x) + 2*E*psi(x))/E - 2*psi(x))/hbar^2 == -2*E*m*psi(x)/hbar^2

In [43]:
psi_region[ 0 ].negate_add( Stepper.RIGHT )

E*m*((hbar^2*m*diff(psi(x), x, x) + 2*E*psi(x))/E - 2*psi(x))/hbar^2 + 2*E*m*psi(x)/hbar^2 == 0

In [44]:
# For later

psi_region[ 2 ] = psi_region[ 0 ].clone()

In [45]:
psi_region[ 2 ].last_step()

E*m*((hbar^2*m*diff(psi(x), x, x) + 2*E*psi(x))/E - 2*psi(x))/hbar^2 + 2*E*m*psi(x)/hbar^2 == 0

In [46]:
k_0 = sp.Symbol( "k_0" )

In [47]:
psi_region[ 0 ].rename( k, k_0 )

AttributeError: 'sage.symbolic.expression.Expression' object has no attribute 'atoms'

In [ ]:
diff_sol = sp.solvers.ode.dsolve( psi_region[ 0 ].last_step().lhs, 0, ivar = x )
#ics = { psi( 0 ): 0, psi( well_length ): 0 }

In [ ]:
diff_sol

In [ ]:
psi_region[ 0 ].add_step( diff_sol )

In [ ]:
C1 = sp.Symbol( "C1" )
C2 = sp.Symbol( "C2" )
constants = [ [ sp.Symbol( 'A' ), sp.Symbol( 'B' ) ] ]

psi_region[ 0 ].rename( C1, constants[ 0 ][ 0 ] )

In [ ]:
psi_region[ 0 ].rename( C2, constants[ 0 ][ 1 ] )

In [ ]:
psi_region[ 1 ].last_step()

In [ ]:
psi_region[ 0 ].last_step()

In [ ]:
psi_region[ 1 ].negate_add( Stepper.RIGHT )

In [ ]:
psi_region[ 1 ].manipulate( lambda side : side.collect( psi(x) ) )

In [ ]:
psi_region[ 1 ].last_step().lhs.as_two_terms()[ 1 ]

In [ ]:
region_1_clone = psi_region[ 1 ].clone()

region_1_clone.manipulate( lambda side : side 
                - region_1_clone.last_step()
                        .lhs.as_two_terms()[ 1 ]
        )

In [ ]:
region_1_clone.manipulate( lambda side : side / region_1_clone.last_step().lhs.as_two_terms()[ 0 ] )

In [ ]:
region_1_clone.manipulate( lambda side : side / region_1_clone.last_step().rhs )

In [ ]:
region_1_clone.manipulate( lambda side : side * sp.Derivative( psi( x ), ( x, 2 ) ) )

In [ ]:
region_1_clone.manipulate( lambda side : side / psi( x ) )

In [ ]:
k_1 = sp.Symbol( "k_1" )
k_1_squared = sp.Eq( k_1 ** 2, region_1_clone.last_step().lhs )

In [ ]:
k_1_squared

In [ ]:
region_1_clone.manipulate( lambda side : side.subs( { k_1_squared.rhs : k_1_squared.lhs } ) )

In [ ]:
region_1_clone.manipulate( lambda side : side * psi( x ) )

In [ ]:
region_1_clone.negate_add( Stepper.RIGHT )

In [ ]:
psi_region[ 1 ] = region_1_clone

In [ ]:
psi_region[ 1 ].last_step()

In [ ]:
psi_1 = sp.Function( "psi_1" )

In [ ]:
psi_2 = sp.Function( "psi_2" )

In [ ]:
psi_region[ 1 ].rename( psi( x ), psi_1( x ) )

In [ ]:
psi_region[ 1 ].add_step( sp.solvers.ode.dsolve( psi_region[ 1 ].last_step() ) )

In [ ]:
constants.append( [ sp.Symbol( 'C' ), sp.Symbol( 'D' ) ] )

In [ ]:
psi_region[ 1 ].rename( C1, constants[ 1 ][ 0 ] )

In [ ]:
psi_region[ 1 ].rename( C2, constants[ 1 ][ 1 ] )

In [ ]:
k_2 = sp.Symbol( "k_2" )
psi_2_symbol = sp.Function( "psi_2" )

psi_region[ 2 ].rename( psi( x ), psi_2_symbol( x ) )

In [ ]:
psi_region[ 2 ].rename( k, k_2 )

In [ ]:
psi_region[ 2 ].add_step( sp.solvers.ode.dsolve( psi_region[ 2 ].last_step() ) )

In [ ]:
constants.append( [ sp.Symbol( 'G' ) ] ) #Skipping a letter in the alphabet so there is no confusion with energy

psi_region[ 2 ].rename( C2, constants[ 2 ][ 0 ] )

In [ ]:
psi_region[ 2 ].rename( C1, 0 )

In [ ]:
#diff_sol = sp.solvers.ode.dsolve( psi_region[ 1 ].last_step().lhs, 0, ivar = x, 
#        ics = { 
#                psi_1( 0 ) : psi( 0 ), 
#                psi_1( barrier_length ) : psi_2( barrier_length ), 
#                psi_1( x ).diff( x, 1 ).subs( x, 0 ) : psi( x ).diff( x, 1 ).subs( x, 0 ), 
#                psi_1( x ).diff( x, 1 ).subs( x, barrier_length ) : psi_2( x ).diff( x, 1 ).subs( x, barrier_length )
#        }
#    )

In [ ]:
flat_constants = []
for constant_set in constants: 
    flat_constants += constant_set


normalize = lambda expression, from_, to : sp.integrate( expression * sp.conjugate( expression ), ( x, from_, to ) )





constants_helpers = sp.solve( 
        [ equation.last_step() for equation in psi_region ], 
        flat_constants
    )

In [ ]:
constants_helpers[ constants[ 0 ][ 0 ] ]

In [ ]:
constants_helpers[ constants[ 1 ][ 0 ] ]

In [ ]:
constants_helpers[ constants[ 2 ][ 0 ] ]

In [ ]:
psi_0_integral = normalize( psi_region[ 0 ].last_step().rhs, 0, TunnelPotential.DEFAULT_WELL_LENGTH )

In [ ]:
psi_0_integral

In [ ]:
psi_region[ 0 ].last_step().rhs * sp.conjugate( psi_region[ 0 ].last_step().rhs )

In [ ]:
aaa = psi_0_integral.simplify().refine( sp.Q.real( constants[ 0 ][ 1 ] ) )

In [ ]:
aaa = aaa.refine( sp.Q.real( constants[ 0 ][ 0 ] ) )

In [ ]:
aaa = aaa.refine( sp.Q.real( k_0 ) )

In [ ]:
bbb = =aaa.trigsimp().simplify().collect( constants[ 0 ][ 0 ] ).collect( constants[ 0 ][ 1 ] ).collect( k_0 ).as_real_imag()[ 0 ].refine( sp.Q.real( constants[ 0 ][ 0 ] ) ).refine( sp.Q.real( k_0 ) ).refine( sp.Q.real( constants[ 0 ][ 1 ] ) )